In [1]:
#import libraries
import os
import cv2
import numpy as np
import keras
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D, Activation, Multiply, Conv2D, Concatenate, Flatten
from keras.models import Model
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from numpy.random import seed

seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
tf.config.experimental.enable_op_determinism()

2023-02-17 12:53:51.844997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 12:53:52.039404: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cserv1_a/soc_ug/fy19iars/project/lib/python3.9/site-packages/cv2/../../lib64:/uollinapps/AppsData/src/vscode/1.71.2-1663191299.el7/lib64
2023-02-17 12:53:52.039437: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 12:53:54.654987: W tensorflow/compiler/xla/stream_executor/platform/defa

Helper functions and variables

In [2]:
def list_labels(file):
    labels_file = open(file, "r")
    labels = []
    
    for line in labels_file:
        label = line.strip()
        labels.append(label)
    
    labels_file.close()
    
    return labels


#list of all labels
class_names = list_labels("./CamSDD/Labels.txt")
class_name_labels = {class_name:i for i, class_name in enumerate(class_names)}



def load_data(folder):
    Category = ["training", "test", "validation"]
    output = []
    
    for category in Category:
        print("Loading {}".format(category))
        path = os.path.join(folder, category)
        print(path)
        images = []
        labels = []
        
        for sub_folder in os.listdir(path):
            label = class_name_labels[sub_folder]
            
            #Iterating through all images
            for file in os.listdir(os.path.join(path, sub_folder)):
                
                #getting the image path
                img_path = os.path.join(os.path.join(path, sub_folder), file)
                
                #appending image and corresponding label
                images.append(cv2.resize(cv2.imread(img_path), (224, 224)))
                # images.append(cv2.imread(img_path))
                labels.append(label)
            
        #check that data type doesn't affect accuracy
        images = np.array(images, dtype='float32')/255
        labels = np.array(labels, dtype='int8')
        
        output.append((images, labels))
        
    return output



#displays 25 images with labels
def display_examples(class_names, images, labels):
    figsize = (20, 20)
    fig = plt.figure(figsize=figsize)
    fig.suptitle("Example of images", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # image = cv2.resize(images[i], figsize)
        plt.imshow(images[i].astype(np.uint8))
        plt.xlabel(class_names[labels[i]])
    plt.show()

Loading data

In [3]:
(train_images, train_labels), (test_images, test_labels), (validation_images, validation_labels)= load_data("./CamSDD")

Loading training
./CamSDD/training
Loading test
./CamSDD/test
Loading validation
./CamSDD/validation


In [4]:
#shuffling train data
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
validation_images, validation_labels = shuffle(validation_images, validation_labels, random_state=25)

Visualize data

In [ ]:
display_examples(class_names, train_images, train_labels)

Creating model

In [ ]:
#channel module
def channel_attention_module(x, ratio=16):
    b, _, _, channel = x.shape
    # MLP shared layer
    l1 = Dense(channel//ratio, activation="relu", use_bias=True)
    l2 = Dense(channel, use_bias=False)
    
    # Global Average pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)
    
    # Global Max pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)
    
    # Adding both and applying sigmoid
    features = x1 + x2
    features = Activation("sigmoid")(features)
    features = Multiply()([x, features])
    
    return features

# spatial attention module
def spatial_attention_module(x):
    # Average pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)
    
    # Max pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    
    # Concatenate
    features = Concatenate()([x1, x2])
    
    # Conv layer
    features = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(features)
    features = Multiply()([x, features])
    
    return features
    
# CBAM
def CBAM(x):
    x = channel_attention_module(x)
    x = spatial_attention_module(x)
    return x

In [5]:
base_model=MobileNet(weights='imagenet', include_top=False) 

x=base_model.output

x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='sigmoid')(x)
x=Dropout(0.7)(x)
output = Dense(30, activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

2023-02-17 12:55:10.759414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cserv1_a/soc_ug/fy19iars/project/lib/python3.9/site-packages/cv2/../../lib64:/uollinapps/AppsData/src/vscode/1.71.2-1663191299.el7/lib64
2023-02-17 12:55:10.759467: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-17 12:55:10.759567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feng-linux-04.leeds.ac.uk): /proc/driver/nvidia/version does not exist
2023-02-17 12:55:10.760003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:

In [ ]:
#checking model architecture
# print(model.summary())
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=1, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history = model.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor], shuffle=False)

Epoch 1/100
495/495 [==============================] - 55s 107ms/step - loss: 2.3517 - accuracy: 0.3938 - val_loss: 1.1229 - val_accuracy: 0.8300
Epoch 2/100
495/495 [==============================] - 52s 106ms/step - loss: 1.0649 - accuracy: 0.7365 - val_loss: 0.6769 - val_accuracy: 0.8650
Epoch 3/100
495/495 [==============================] - 52s 105ms/step - loss: 0.7460 - accuracy: 0.8040 - val_loss: 0.5281 - val_accuracy: 0.8783
Epoch 4/100
495/495 [==============================] - 51s 102ms/step - loss: 0.5941 - accuracy: 0.8360 - val_loss: 0.4490 - val_accuracy: 0.8883
Epoch 5/100
495/495 [==============================] - 52s 104ms/step - loss: 0.5123 - accuracy: 0.8588 - val_loss: 0.4039 - val_accuracy: 0.8883
Epoch 6/100
495/495 [==============================] - 51s 104ms/step - loss: 0.4501 - accuracy: 0.8722 - val_loss: 0.3705 - val_accuracy: 0.9000
Epoch 7/100
495/495 [==============================] - 52s 104ms/step - loss: 0.4097 - accuracy: 0.8799 - val_loss: 0.3492 -

Evaluating trained model

In [9]:
# model.save("tmp6")
# model = keras.models.load_model('CBAM_pooling_r16_false')

score = model.evaluate(test_images, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

19/19 [==============================] - 3s 145ms/step - loss: 0.2733 - accuracy: 0.9167
Test loss: 0.27327215671539307
Test accuracy: 0.9166666865348816


In [ ]:
def plot_accuracy_loss(history):
    # #plot accuracy
    # plt.subplot(221)
    # plt.plot(history.history["accuracy"], 'bo--', label="acc")
    # plt.plot(history.history["val_accuracy"], "ro--", label = "val_acc")
    # plt.title("train_acc vs val_acc")
    # plt.ylabel("accuracy")
    # plt.xlabel("epochs")
    # plt.legend()
    
    #plot loss function
    plt.subplot(221)
    plt.plot(history.history["loss"], 'bo--', label="loss")
    plt.plot(history.history["val_loss"], "ro--", label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.legend()
    plt.show()

plot_accuracy_loss(history)

In [ ]:
predictions = model.predict(test_images)
pred_labels = np.argmax(predictions, axis=1)
print(classification_report(test_labels, pred_labels))